In [9]:
# Tutorial: https://youtu.be/rc_Y6rdBqXM
###!pip install binance
#!pip install python-binance

In [49]:
# Binance, including sockets
from binance import BinanceSocketManager
from binance.client import Client
from binance import Client#, ThreadedWebsocketManager, ThreadedDepthCacheManager
import datetime as dt

from secretttfolder import keystestnet

# Standart libraries
import pandas as pd
import numpy as np

import time

import h5py
import pyarrow.parquet as pq


# API keys:
api_key = keystestnet.api_key
api_secret = keystestnet.api_secret
client = Client(api_key, api_secret, testnet=True)
client.API_URL = 'https://testnet.binance.vision/api'

# Settings:
symb = "BTCUSDT"
interval = "5MINUTE"

In [30]:
candles = client.get_klines(symbol=symb, interval=Client.KLINE_INTERVAL_5MINUTE)
numpycandles = np.array(candles)

# reshape date to pandas
df = pd.DataFrame(numpycandles.reshape(-1, 12), dtype=float, columns=('Open Time','Open','High','Low','Close','Volume','Close time','Quote asset volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore'))
df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')

# convert_to_hdf5 = df.to_hdf('prices.h5', key='binance_prices', mode='a')

close_prices = df['Close']

# convert pandas dataframe to h5py
with h5py.File('prices.h5', 'a') as hf:
    hf.create_dataset('close_prices', data=close_prices)

In [48]:
# write the DataFrame to a parquet file
df.to_parquet('example.parquet')

In [50]:
# Read the Parquet file as a Pandas DataFrame
df2 = pd.read_parquet('example.parquet')

# Print the first 5 rows
print(df2.head(5))

          axis0  axis1 block0_items  \
0  b'Open Time'      0      b'Open'   
1       b'Open'      1      b'High'   
2       b'High'      2       b'Low'   
3        b'Low'      3     b'Close'   
4      b'Close'      4    b'Volume'   

                                       block0_values  block1_items  \
0  [27343.96, 27353.79, 27321.37, 27345.72, 20.43...  b'Open Time'   
1  [27345.72, 27352.58, 27331.6, 27352.36, 14.813...          None   
2  [27352.35, 27369.63, 27351.09, 27369.38, 14.68...          None   
3  [27369.38, 27376.08, 27367.34, 27369.74, 9.514...          None   
4  [27369.97, 27388.11, 27369.83, 27385.74, 13.47...          None   

           block1_values  
0  [1682394900000000000]  
1  [1682395200000000000]  
2  [1682395500000000000]  
3  [1682395800000000000]  
4  [1682396100000000000]  


In [39]:
"""# Print only Open column from hdf5 file
# To print only the "Open" column from an HDF5 file, you can use the h5py library in Python. Here's an example code snippet:
 
# Open the HDF5 file in read-only mode
with h5py.File('prices.h5', 'r') as f:
    # Get the dataset containing the data
    dataset = f['binance_prices']

    # Get the names of all columns in the dataset
    column_names = list(dataset.keys())

    # Print the first 5 rows of each column
    for name in column_names:
        print(name)
        print(dataset[name][0:5])

"""

axis0
[b'Open Time' b'Open' b'High' b'Low' b'Close']
axis1
[0 1 2 3 4]
block0_items
[b'Open' b'High' b'Low' b'Close' b'Volume']
block0_values
[[2.73439600e+04 2.73537900e+04 2.73213700e+04 2.73457200e+04
  2.04305140e+01 1.68239520e+12 5.58572502e+05 5.46000000e+02
  9.99235600e+00 2.73209627e+05 0.00000000e+00]
 [2.73457200e+04 2.73525800e+04 2.73316000e+04 2.73523600e+04
  1.48136520e+01 1.68239550e+12 4.05039223e+05 3.77000000e+02
  8.47492900e+00 2.31731822e+05 0.00000000e+00]
 [2.73523500e+04 2.73696300e+04 2.73510900e+04 2.73693800e+04
  1.46806560e+01 1.68239580e+12 4.01636404e+05 3.67000000e+02
  1.08739010e+01 2.97491840e+05 0.00000000e+00]
 [2.73693800e+04 2.73760800e+04 2.73673400e+04 2.73697400e+04
  9.51475900e+00 1.68239610e+12 2.60428984e+05 2.38000000e+02
  5.95587200e+00 1.63017555e+05 0.00000000e+00]
 [2.73699700e+04 2.73881100e+04 2.73698300e+04 2.73857400e+04
  1.34761890e+01 1.68239640e+12 3.68977592e+05 3.48000000e+02
  8.81236300e+00 2.41280634e+05 0.00000000e+00

In [46]:
""" 
# Open the HDF5 file in read-only mode
with h5py.File('prices.h5', 'r') as f:
    # Get the dataset containing the data
    dataset = f['binance_prices']

    # Get the names of all columns in the dataset
    column_names = list(dataset.keys())

    # Create a dictionary to store the data
    data_dict = {}

    # Store the first 5 rows of each column in the dictionary
    for name in column_names:
        data_dict[name] = dataset[name][0:5]

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(data_dict, orient='index').T

# Print the DataFrame
print(df)
"""

          axis0 axis1 block0_items  \
0  b'Open Time'     0      b'Open'   
1       b'Open'     1      b'High'   
2       b'High'     2       b'Low'   
3        b'Low'     3     b'Close'   
4      b'Close'     4    b'Volume'   

                                       block0_values  block1_items  \
0  [27343.96, 27353.79, 27321.37, 27345.72, 20.43...  b'Open Time'   
1  [27345.72, 27352.58, 27331.6, 27352.36, 14.813...          None   
2  [27352.35, 27369.63, 27351.09, 27369.38, 14.68...          None   
3  [27369.38, 27376.08, 27367.34, 27369.74, 9.514...          None   
4  [27369.97, 27388.11, 27369.83, 27385.74, 13.47...          None   

           block1_values  
0  [1682394900000000000]  
1  [1682395200000000000]  
2  [1682395500000000000]  
3  [1682395800000000000]  
4  [1682396100000000000]  


In [12]:
# Define the RSI indicator
def rsi(price, period=14):
    delta = price.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Calculate the RSI indicator
rsi_indicator = rsi(df['Close'])


# Define Divergence 
def divergence(price_set, rsi_window=14, fast_ma = 1, slow_ma = 50):

# Calculate the RSI indicator for using in calculation of divergence 
   rsi_indicator = rsi(price_set)

# Calculate the Divergence indicator. 
   bullish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() > 0))
   bearish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() < 0))

   fast_ma_df = df[f'ma{fast_ma}'] = df['Close'].rolling(window=fast_ma).mean()
   slow_ma_df = df[f'ma{slow_ma}'] = df['Close'].rolling(window=slow_ma).mean()
   trend_up = fast_ma_df > slow_ma_df
   trend_down = fast_ma_df < slow_ma_df
   
   # Combine all previous indicators into the one 
   # We need that if bullish_divergence (trend_up), True PLUS fast MA above slow MA, then set 1, everything else - 0. If bearish_divergence True, PLUS fast MA above slow MA (trend_down), then set -1
   indicator = np.where( ( ( (bullish_divergence) == True)  & (trend_up == True) ), 1, 0)
   indicator = np.where( ( ( (bearish_divergence) == True)  & (trend_down == True) ), -1, indicator)
   
   return indicator

diver = divergence(close_prices, rsi_window=14, fast_ma = 1, slow_ma = 50)
last_indicator_value = diver[-1]

In [13]:
last_indicator_value

0

In [14]:
last_indicator_value = 1

In [15]:
last_indicator_value

1

In [16]:
def trade(ticker, lotsize, open_position = False):
  bm = BinanceSocketManager(client)
  trade_socket = bm.trade_socket(ticker)
  print(dt.datetime.now())
  if last_indicator_value > 0 and last_indicator_value != 0 and not open_position:
    order = client.create_order(symbol=ticker,
                                side='BUY',
                                type='MARKET',
                                quantity=lotsize)
    print(order)
    buyprice = float(order['fills'][0]['price'])
    open_position = True
    return
  if open_position and last_indicator_value < 0 and last_indicator_value != 0:
      order = client.create_order(symbol=ticker,
                                side='SELL',
                                type='MARKET',
                                quantity=lotsize)
      print(order)
      return
  time.sleep(1)  # sleep 1 seconds
  
trade('BTCUSDT', 0.001)

2023-04-27 00:09:35.424656
{'symbol': 'BTCUSDT', 'orderId': 11439939, 'orderListId': -1, 'clientOrderId': 'OGmAhXBDpt56sW0i5RIG9T', 'transactTime': 1682543376264, 'price': '0.00000000', 'origQty': '0.00100000', 'executedQty': '0.00100000', 'cummulativeQuoteQty': '28.27804000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1682543376264, 'fills': [{'price': '28278.04000000', 'qty': '0.00100000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'tradeId': 3523714}], 'selfTradePreventionMode': 'NONE'}


In [20]:
info = client.get_account()
info

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1682543376264,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00100000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9971.71808000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [18]:
def trade(ticker, lotsize, open_position = False):
  bm = BinanceSocketManager(client)
  trade_socket = bm.trade_socket(ticker)
  print(dt.datetime.now())
  if last_indicator_value > 0 and last_indicator_value != 0 and not open_position:
    order = client.create_order(symbol=ticker,
                                side='BUY',
                                type='MARKET',
                                quantity=lotsize,
                                timeInForce='GTC')
    print(order)
    buyprice = float(order['fills'][0]['price'])
    open_position = True
    return
  if open_position and last_indicator_value < 0 and last_indicator_value != 0:
      order = client.create_order(symbol=ticker,
                                side='SELL',
                                type='MARKET',
                                quantity=lotsize,
                                timeInForce = 'GTC')
      print(order)
      return
  time.sleep(1)  # sleep 1 seconds
  
trade('BTCUSDT', 0.0001)

2023-04-27 00:09:37.160219


BinanceAPIException: APIError(code=-1106): Parameter 'timeInForce' sent when not required.

In [ ]:
"""1. Check balance
2. Check if price of current candle is not differ by x% from price from other exchange
3. Check if Ta-Lib indicator value not differ more than y% from another exchange
4. Check open positions
5. Check id position already open
6. Open position characteristics: long/short, timeframe, symbol, indicators values. Are any other positions with same characteristics? If yes, than it's duplicate 
7. Before Closing: check position
8. After closing: position closed? Ok, go check if we have opposite signal. We have opposite signal? Ok, go open opposite Trade Position
9. Position open time
10. How many bars passed
11. Get order ID
"""